# Функции

## Задача 1
### Пункт 1

In [1]:
def owner_by_document_id(**kwargs):
    doc_id = input('Введите номер документа: ')
    owner = None
    
    for doc in kwargs['documents']:
        if doc['number'] == doc_id:
            owner = doc['name']
            break
    if owner:
        action_text = f'Владелец документа: {owner}'
    else:
        action_text = 'Документ не найден в базе'
    
    print(action_text)

### Пункт 2

In [2]:
def shelf_by_document_id(*args, **kwargs):
    doc_id = args[0] if args else input('Введите номер документа: ')
    
    shelf = None
    for shelf_id, directory in kwargs['directories'].items():
        if doc_id in directory:
            shelf = shelf_id
            break
    if shelf is not None:
        action_text = f'Документ хранится на полке: {shelf[0]}'
    else:
        action_text = 'Документ не найден в базе'
    
    if 'logs_off' not in kwargs:
        print(action_text)
    
    return shelf 

### Пункт 3

In [3]:
def documents_full_info(**kwargs):
    for doc in kwargs['documents']:
        kwargs.setdefault('logs_off')
        doc_shelf = shelf_by_document_id(doc['number'], **kwargs)
        print(f'№: {doc["number"]}, тип: {doc["type"]}, владелец: {doc["name"]}, полка хранения: {doc_shelf}')

### Пункт 4

In [4]:
def add_new_shelf(**kwargs):
    new_shelf = input('Введите номер полки: ')

    if new_shelf not in kwargs['directories']:
        kwargs['directories'][new_shelf] = []
        action_text = 'Полка добавлена.'
    else:
        action_text = 'Такая полка уже существует.'

    common_text = f'Текущий перечень полок: {", ".join(kwargs["directories"])}'
    print(action_text, common_text)

### Пункт 5

In [5]:
def del_shelf(**kwargs):
    shelf_for_del = input('Введите номер полки: ')

    if shelf_for_del in kwargs['directories']:
        if kwargs['directories'][shelf_for_del]:
            action_text = 'На полке есть документы, удалите их перед удалением полки.'
        else:
            del kwargs['directories'][shelf_for_del]
            action_text = 'Полка удалена.'
    else:
        action_text = 'Такой полки не существует.'

    common_text = f'Текущий перечень полок: {", ".join(kwargs["directories"])}'
    print(action_text, common_text)

## Задача 2
### Пункт 1

In [14]:
# Вспомогательная функция
def get_document_index(doc_id, **kwargs):
    index = -1
    for pos, doc in enumerate(kwargs['documents']):
        if doc['number'] == doc_id:
            index = pos
            break
    return index

In [15]:
def add_new_document(**kwargs):
    # Рациональнее сначала запрашивать полку, чтобы не заставлять
    # пользователя повторно вводить данные в случае ее отсутствия
    shelf = input('Введите полку для хранения: ')
    
    if shelf not in kwargs['directories']:
        action_text = 'Такой полки не существует. Добавьте полку командой as.'
    else:
        doc_id = input('Введите номер документа: ')
        doc_type = input('Введите тип документа: ')
        doc_owner = input('Введите владельца документа: ')
        new_doc = {'type': doc_type, 'number': doc_id, 'name': doc_owner}
        # Проверяем, не дубль ли это
        duplicate = False
        doc_indx = get_document_index(doc_id, **kwargs)
        if doc_indx > -1:
            duplicate = True
            for key in new_doc.keys():
                if new_doc[key] != kwargs['documents'][doc_indx][key]:
                    duplicate = False
                    break
        if duplicate == True:
            action_text = 'Такой документ уже существует.'
        else:
            kwargs['documents'].append(new_doc)
            kwargs['directories'][shelf].append(doc_id)
            action_text = 'Документ добавлен.'
        
    common_text = 'Текущий список документов:'
    print(action_text, common_text, sep='\n')
    documents_full_info(**kwargs)

### Пункт 2

In [8]:
def del_document(**kwargs):
    doc_id = input('Введите номер документа: ')
    doc_index = get_document_index(doc_id, **kwargs)
    if doc_index > -1:
        # 1. Удаляем документ с "полки"
        doc_shelf = shelf_by_document_id(doc_id, logs_off=True, **kwargs)
        kwargs['directories'][doc_shelf].remove(doc_id)
        # 2. Удаляем документ из списка документов
        del kwargs['documents'][doc_index]
        action_text = 'Документ удален.'
    else:
        action_text = 'Документ не найден в базе.'
    
    common_text = 'Текущий список документов:'
    print(action_text, common_text, sep='\n')
    documents_full_info(**kwargs)

### Пункт 3

In [9]:
def move_document(**kwargs):
    doc_id = input('Введите номер документа: ')
    new_shelf = input('Введите номер полки: ')
    
    if get_document_index(doc_id, **kwargs) > -1:
        if new_shelf in kwargs['directories']:
            old_shelf = shelf_by_document_id(doc_id, logs_off=True, **kwargs)
            kwargs['directories'][old_shelf].remove(doc_id)
            kwargs['directories'][new_shelf].append(doc_id)
            action_text = 'Документ перемещен.'
        else:
            action_text = 'Такой полки не существует.'
    else:
        action_text = 'Документ не найден в базе.'
    
    if 'документ' in action_text.lower():
        common_text = 'Текущий список документов:'
        print(action_text, common_text, sep='\n')
        documents_full_info(**kwargs)
    else:
        common_text = f'Текущий перечень полок: {", ".join(kwargs["directories"])}'
        print(action_text, common_text)

### Главная функция

In [10]:
def secretary_assistant(documents, directories):
    all_commands = {
        'p': owner_by_document_id,
        's': shelf_by_document_id,
        'l': documents_full_info,
        'as': add_new_shelf,
        'ds': del_shelf,
        'ad': add_new_document,
        'd': del_document,
        'm': move_document
    }
    while True:
        command = input('Введите команду: ')
        if command == 'q':
            return
        else:
            try:          
                all_commands[command](documents=documents, directories=directories)
            except:
                print('Вы ввели неверную команду')

## Проверка

In [12]:
# Проверка примеров из задания
documents = [
    {'type': 'passport', 'number': '2207 876234', 'name': 'Василий Гупкин'},
    {'type': 'invoice', 'number': '11-2', 'name': 'Геннадий Покемонов'},
    {'type': 'insurance', 'number': '10006', 'name': 'Аристарх Павлов'}
]
directories = {
    '1': ['2207 876234', '11-2'],
    '2': ['10006'],
    '3': []
}
secretary_assistant(documents, directories)

Введите команду: p
Введите номер документа: 10006
Владелец документа: Аристарх Павлов
Введите команду: p
Введите номер документа: 12345
Документ не найден в базе
Введите команду: s
Введите номер документа: 10006
Документ хранится на полке: 2
Введите команду: s
Введите номер документа: 12345
Документ не найден в базе
Введите команду: l
№: 2207 876234, тип: passport, владелец: Василий Гупкин, полка хранения: 1
№: 11-2, тип: invoice, владелец: Геннадий Покемонов, полка хранения: 1
№: 10006, тип: insurance, владелец: Аристарх Павлов, полка хранения: 2
Введите команду: as
Введите номер полки: 10
Полка добавлена. Текущий перечень полок: 1, 2, 3, 10
Введите команду: as
Введите номер полки: 1
Такая полка уже существует. Текущий перечень полок: 1, 2, 3, 10
Введите команду: ds
Введите номер полки: 3
Полка удалена. Текущий перечень полок: 1, 2, 10
Введите команду: ds
Введите номер полки: 1
На полке есть документы, удалите их перед удалением полки. Текущий перечень полок: 1, 2, 10
Введите команду:

In [17]:
# Проверка обработки дублей
documents = [
    {'type': 'passport', 'number': '2207 876234', 'name': 'Василий Гупкин'},
    {'type': 'invoice', 'number': '11-2', 'name': 'Геннадий Покемонов'},
    {'type': 'insurance', 'number': '10006', 'name': 'Аристарх Павлов'}
]
directories = {
    '1': ['2207 876234', '11-2'],
    '2': ['10006'],
    '3': []
}
secretary_assistant(documents, directories)

Введите команду: ad
Введите полку для хранения: 2
Введите номер документа: 10006
Введите тип документа: insurance
Введите владельца документа: Аристарх Павлов
Такой документ уже существует.
Текущий список документов:
№: 2207 876234, тип: passport, владелец: Василий Гупкин, полка хранения: 1
№: 11-2, тип: invoice, владелец: Геннадий Покемонов, полка хранения: 1
№: 10006, тип: insurance, владелец: Аристарх Павлов, полка хранения: 2
Введите команду: q
